https://docs.aws.amazon.com/glue/latest/dg/release-notes.html <BR>
AWS Glue interactive sessions are not yet available in AWS Glue 4.0.    

In [7]:
%glue_version 3.0
%number_of_workers 2
%worker_type G.2X
%idle_timeout 6000
%region ap-northeast-2
%iam_role arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Setting Glue version to: 3.0
Previous number of workers: 5
Setting new number of workers to: 2
Previous worker type: G.1X
Setting new worker type to: G.2X
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 6000 minutes.
Previous region: None
Setting new region to: ap-northeast-2
Region is set to: ap-northeast-2
Current iam_role is None
iam_role has been set to arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter.


glue 카탈로그에 sample2 라는 데이터베이스를 생성한다.

In [1]:
import boto3

glueClient = boto3.client('glue')

response = glueClient.create_database(
    DatabaseInput={
        'Name': 'sample2'
    }
)                                                    
    

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 2
Session ID: 3a06baf1-9866-4602-ab81-74fb8191329d
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--enable-glue-datacatalog true
Waiting for session 3a06baf1-9866-4602-ab81-74fb8191329d to get into ready status...
Session 3a06baf1-9866-4602-ab81-74fb8191329d has been created.



스파크 SQL 테이블 스키마 조회 <BR>
https://spark.apache.org/docs/3.0.0-preview/sql-ref-syntax-aux-describe-table.html

In [1]:
%%sql 
describe table EXTENDED sample.`tbl-csv`

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 2
Session ID: 7dda530c-90a3-4a04-b84b-2a0a60b16dd0
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--enable-glue-datacatalog true
Waiting for session 7dda530c-90a3-4a04-b84b-2a0a60b16dd0 to get into ready status...
Session 7dda530c-90a3-4a04-b84b-2a0a60b16dd0 has been created.
+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                year|   bigint|   null|
|             quarter|   bigint|   null|
|               month|   bigint|   null|
|        day_of_month|   bigint|   null|
|         day_of_week|   bigint|   null|
|             fl_date|   string|   null|
|      unique_carrier|   string|   null|
|          airline_id|   bigint|   null|
|             carrier

In [3]:
%%sql 
select year, quarter, month, day_of_month from sample.`tbl-csv` limit 5


+----+-------+-----+------------+
|year|quarter|month|day_of_month|
+----+-------+-----+------------+
|null|   null| null|        null|
|2016|      3|    8|           3|
|2016|      3|    8|           3|
|2016|      3|    8|           3|
|2016|      3|    8|           3|
+----+-------+-----+------------+


In [2]:
%%sql
select count(1) from sample.`tbl-csv`

+--------+
|count(1)|
+--------+
| 5248451|
+--------+


스파크 데이터프레임을 이용한 데이터 조작

In [3]:
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [4]:
%%sql
show tables in sample

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|  sample|        tbl-csv|      false|
|  sample|tbl-flight-part|      false|
+--------+---------------+-----------+


https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-dynamic-frame.html

In [5]:
flightDyf = glueContext.create_dynamic_frame.from_catalog(
    database = "sample",
    table_name = "tbl-csv"
)
flightDyf.printSchema()

root
|-- year: long
|-- quarter: long
|-- month: long
|-- day_of_month: long
|-- day_of_week: long
|-- fl_date: string
|-- unique_carrier: string
|-- airline_id: long
|-- carrier: string
|-- tail_num: string
|-- fl_num: long
|-- origin_airport_id: long
|-- origin_airport_seq_id: long
|-- origin_city_market_id: long
|-- origin: string
|-- origin_city_name: string
|-- origin_state_abr: string
|-- origin_state_fips: long
|-- origin_state_nm: string
|-- origin_wac: long
|-- dest_airport_id: long
|-- dest_airport_seq_id: long
|-- dest_city_market_id: long
|-- dest: string
|-- dest_city_name: string
|-- dest_state_abr: string
|-- dest_state_fips: long
|-- dest_state_nm: string
|-- dest_wac: long
|-- crs_dep_time: long
|-- dep_time: long
|-- dep_delay: long
|-- dep_delay_new: long
|-- dep_del15: long
|-- dep_delay_group: long
|-- dep_time_blk: string
|-- taxi_out: long
|-- wheels_off: long
|-- wheels_on: long
|-- taxi_in: long
|-- crs_arr_time: long
|-- arr_time: long
|-- arr_delay: long
|-- 

In [6]:
flightDyf = flightDyf.drop_fields(
    paths=[
        "origin_airport_id", 
        "origin_airport_seq_id", 
        "origin_city_market_id",
        "origin",
        "origin_city_name",
        "origin_state_abr",
        "origin_state_fips",
        "origin_state_nm",
        "origin_wac",
        "dest_airport_id",
        "dest_airport_seq_id",
        "dest_city_market_id",
        "dest",
        "dest_city_name",
        "dest_state_abr",
        "dest_state_fips",
        "dest_state_nm",
        "dest_wac"
    ]
)

flightDyf.printSchema()
flightDyf.toDF().show()

root
|-- year: long
|-- quarter: long
|-- month: long
|-- day_of_month: long
|-- day_of_week: long
|-- fl_date: string
|-- unique_carrier: string
|-- airline_id: long
|-- carrier: string
|-- tail_num: string
|-- fl_num: long
|-- crs_dep_time: long
|-- dep_time: long
|-- dep_delay: long
|-- dep_delay_new: long
|-- dep_del15: long
|-- dep_delay_group: long
|-- dep_time_blk: string
|-- taxi_out: long
|-- wheels_off: long
|-- wheels_on: long
|-- taxi_in: long
|-- crs_arr_time: long
|-- arr_time: long
|-- arr_delay: long
|-- arr_delay_new: long
|-- arr_del15: long
|-- arr_delay_group: long
|-- arr_time_blk: string
|-- cancelled: long
|-- cancellation_code: string
|-- diverted: long
|-- crs_elapsed_time: long
|-- actual_elapsed_time: long
|-- air_time: long
|-- flights: long
|-- distance: long
|-- distance_group: long
|-- mon: string
|-- carrier_delay: long
|-- weather_delay: long
|-- nas_delay: long
|-- security_delay: long
|-- late_aircraft_delay: long
|-- first_dep_time: long
|-- total_ad

In [7]:
%%sql
select year, month from sample.`tbl-csv`

+----+-----+
|year|month|
+----+-----+
|null| null|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
|2016|    6|
+----+-----+
only showing top 20 rows


In [12]:
s3_bucket = "glue-sink-20230404"
sink = glueContext.getSink(
    path = f"s3://{s3_bucket}/flight/raw-part4/",
    connection_type = "s3",
    updateBehavior = "UPDATE_IN_DATABASE",
    paritionKyes = ["year", "month"],
 #   compress = "snappy",
    enableUpdateCatalog = True,
 #   transformation_ctx = "s3output"
)

sink.setCatalogInfo(catalogDatabase="sample", catalogTableName="tbl-flight4")
sink.setFormat("glueparquet")
sink.writeFrame(flightDyf)


In [13]:
%%sql
show tables in `sample`

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|  sample|        tbl-csv|      false|
|  sample|tbl-flight-part|      false|
|  sample|    tbl-flight4|      false|
+--------+---------------+-----------+


https://docs.aws.amazon.com/glue/latest/dg/aws-glue-programming-etl-partitions.html <BR>
getSink 를 이용하여 S3에 데이터를 저장하는 경우 paritionKeys 를 설정하더라도 파티셔닝이 되지 않는다.
아래의 코드를 사용해야 파티션키에 대한 파티셔닝 처리가 된다. (하지만 카탈로그는 수동으로 업데이트 해야 하는가?)

In [11]:
glueContext.write_dynamic_frame.from_options(
    frame = flightDyf,
    connection_type = "s3",
    connection_options = { 
        "path": f"s3://{s3_bucket}/flight/raw-part3/",
        "partitionKeys": ["year", "month"]
    },
    format = "parquet"
)

In [15]:
# execute glue crawler

flightDyf.getNumPartitions()

23


https://repost.aws/knowledge-center/glue-job-output-large-files 
스파크가 출력하는 파티션의 갯수를 23에서 5개로 변경한다. 워커노드 대비 파티션 갯수를 비례적으로 설정해야 한다.
파티션의 갯수와 크기는 성능에 영향을 끼친다.

In [18]:
flgithDyf_p5 = flightDyf.coalesce(5)
flgithDyf_p5.getNumPartitions()

s3_bucket = "glue-sink-20230404"
sink = glueContext.getSink(
    path = f"s3://{s3_bucket}/flight/raw-p5/",
    connection_type = "s3",
    updateBehavior = "UPDATE_IN_DATABASE",
 #   paritionKyes = ["year", "month"],
 #   compress = "snappy",
    enableUpdateCatalog = True,
 #   transformation_ctx = "s3output"
)

sink.setCatalogInfo(catalogDatabase="sample", catalogTableName="tbl-flight-p5")
sink.setFormat("glueparquet")
sink.writeFrame(flgithDyf_p5)